<a href="https://colab.research.google.com/github/mohitiitr/bb84piby8basis/blob/master/BB84piby8basis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install qiskit

# BB84 Quantum Key Distribution (QKD) Protocol Using Basis (pi/8,0)and(5pi/8,0). 
 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
This file discuss the case without evesdroppers.

The basis are given name as M basis .

First is $cos (\pi/8) \ket{0} +sin(\pi/8)\ket{1}$ denoted by everywhere further by $ \ket{m1/8}$.

Second is  $cos (5\pi/8) \ket{0} +sin(5\pi/8)\ket{1}$ denoted by everywhere further by $ \ket{m5/8}$.

[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 

$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [3]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ

New Section
from qiskit.tools.monitor import job_monitor
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Visualing States of M basis 

In this section there is the code to help us see the visual form of our considered M basis on Bloch Sphere.

In [0]:
BasicAer.backends()
backend = BasicAer.get_backend('statevector_simulator')

In [0]:
m18 = QuantumRegister (1)
c18 = ClassicalRegister(1)
circ2 =QuantumCircuit(m18,c18)
circ2.ry(pi/8, m18)


job1 =execute(circ2,backend)
job_monitor(job1)
result1=job1.result()
chi=result1.get_statevector(circ2)
plot_bloch_multivector(chi)

In [0]:
m58 = QuantumRegister (1)
c58 = ClassicalRegister(1)
circ3 =QuantumCircuit(m58,c58)
circ3.ry(pi/8, m58)


job2 =execute(circ3,backend)
job_monitor(job2)
result2=job2.result()
chi_anti_polar=result2.get_statevector(circ3)
plot_bloch_multivector(chi_anti_polar)


## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

0 $\rightarrow$ Computational basis

1  $\rightarrow$ M basis

The other encodes the state:

0  $\rightarrow \ket{ 0 } or \ket{m1/8} $ 

1  $\rightarrow \ket{ 1 } or \ket{m5/8} $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [0]:
num_qubits = 24 


alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(alice_state)
print(alice_basis)
print(bob_basis)

## Creating the circuit

Based on the following results:

$X\ket{0} = \ket{1}$

$Ry(\pi/8,0)\ket{0} = \ket{m1/8}$


$ Ry(\pi/8,0)X\ket{0} = \ket{m5\pi/8}$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the M basis, she applies an $Ry(\pi/8)$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. Bob measures the qubits according to his binary string. To measure a qubit in the M basis, he applies an $Ry(-\pi/8)$ gate to the corresponding qubit and then performs a mesurement on the computational basis. (Here $-\pi/8$ can be interpreted as $+(2\pi-\pi/8)$  rotation of theta ) 



In [1]:
q = QuantumRegister(num_qubits,'q')
c = ClassicalRegister(num_qubits,'c')

bb84 = QuantumCircuit(q,c,name='bb84')

# Alice prepares her qubits
for index, _ in enumerate(alice_basis):
    if alice_state[index] == 1:
        bb84.x(q[index])
    if alice_basis[index] == 1:
        bb84.ry(pi/8 , q[index])
bb84.barrier()  

# Bob measures the received qubits
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 1:
        bb84.ry(-pi/8 ,q[index])
        
bb84.barrier()        
bb84.measure(q,c)
    
            
bb84.draw()


NameError: ignored

In [0]:
#USE THESE LINES TO RUN THE CODE ON IBMQ MACHINES WITH YOUR IDENTIFICATION TOKEN

# token = ''
# IBMQ.enable_account(token)
# IBMQ.backends()



[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
# backend = IBMQ.get_backend('ibmq_qasm_simulator')

job = execute(bb84,backend,shots=1024)

job_monitor(job)

result = job.result()
counts = result.get_counts()

# Prints out all the measured binary strings by Bob (reversed)
for count in [*counts]:
    print(count) 

Job Status: job has successfully run
110111100111011000011001
110011010110000000000010
010001000111011000000000
010011101111001000010011
010001110111000000001011
011011010110000000010001
010001011110011000001001
011111101110011000001001
011001111110011000011000
111101001110010000011011
010011111110010000011011
111011101110010000000000
011101101110010000010000
011101001111001000000001
110001010110010000011000
010011110111000000011000
010101001111001000010011
010011110110010000011011
111101001110010000011000
010101001110011000001001
011111001111000000000001
010101101110001000001010
110001001110010000011000
111011100111000000011010
110101011110010000011000
011101100110010000010010
110101101110011000011001
111011101110011000001010
111101111111001000001010
110001101110000000001000
111001011111000000000001
011101010110011000010001
110111100111010000010001
011101001110000000001010
011011011111001000010001
010011010110010000011001
110001010110010000001001
011111111111011000000010
0100011111110

In [0]:
for temp_key in [*counts]:
    temp_key_rev = temp_key[::-1] # qubit orders are reversed in qiskit
    key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
             key += str(temp_key_rev[i])
    print(key)

0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111
0000001111